In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn import pipeline
from sklearn import preprocessing

In [5]:
#Loading hiPSC data for our 10 cell lines
lines = ["Bima1", "Kolf2", "Kolf3", "Kucg2", "Letw5", "Podx1", "Qolg1", "Sojd3", "Wibj2", "Yoch6"]
TPMS_reps = pd.read_csv('TPMS_hiPSC.csv', index_col=0)
clusters=pd.read_csv("Supplementary_File_1.csv", index_col=0)

In [7]:
#DEGs in hiPSCs for which H3K27me3 was predictive of transcription.
geneid_K27=clusters[(clusters.cluster=='K27') | (clusters.cluster=='K4&K27') | (clusters.cluster=='K4&K27&ATAC')].gene_id

In [8]:
#Genes missing in GRCh37, which appear in GRCh38
missing=['ENSG00000054598', 'ENSG00000229637', 'ENSG00000273604',
       'ENSG00000275023', 'ENSG00000279692']

In [11]:
# Keeping only H3K27me3-regulated genes
K27_RNA_vals=TPMS_reps[TPMS_reps.geneid.isin(geneid_K27)]

# Excluding genes not present in GRCh37
K27_RNA_vals=K27_RNA_vals[~K27_RNA_vals.geneid.isin(missing)]

In [12]:
# Did the cell lines differentiate into PGCLCs? For each RNA replicate, 
# although success was evaluated at the cell line level.
DP=[1,1,1,1,1,1,0,0,0,0,1,1,1,1,0,0,1,1,0,0,0]

#To flag the least-common outcome: Failed->1
rDP=np.ones(21)-DP
rDP

array([0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0.,
       0., 1., 1., 1.])

In [13]:
#Preparing the data for model.

K27_RNA_vals=K27_RNA_vals.iloc[:,:21]
K27_RNA_valsT=K27_RNA_vals.transpose()

In [14]:
#Preparing the data for bootstraping

Xall=K27_RNA_valsT.to_numpy()
Yall=np.array(rDP)

samplesX = tuple(Xall[:, i] for i in range(Xall.shape[1]))
samples = samplesX + (Yall,)

In [15]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

pipe = pipeline.Pipeline([('scaler', MinMaxScaler()), ('LR', LogisticRegression(solver="liblinear", C=1, random_state=42))])

def coeff_reg_scaled(*samples):
    samples = np.asarray(samples)
    X = samples[:-1].T
    y = samples[-1]
    #C=1, lenient regularisation to explore larger values of coefficients
    clf = pipe.fit(X, y)
    return clf['LR'].coef_

In [16]:
import scipy.stats as sts

#Bootstrapping logisitc regression models, to get a sense of the dispersion of coefficients.
res = sts.bootstrap(samples, statistic=coeff_reg_scaled, n_resamples=9999, paired=True, random_state=24601)

In [17]:
# Keep only genes for which the 95% confidence interval does not overlap with 0
# This is to TRY to keep the most robust predictors.
fcols=np.sign(res.confidence_interval[0][0])==np.sign(res.confidence_interval[1][0])
# 105 genes out of 147, as of 16/08/2025; 109 as of 2/12/2025

In [19]:
#Filter columns in the dataset; keeping robust predictors only.
f_K27_RNA=K27_RNA_valsT[K27_RNA_valsT.columns[fcols]]


In [21]:
#Function for within-sample accuracy.

from sklearn.model_selection import train_test_split

#More strict regularisation to prevent overfitting.
rpipe = pipeline.Pipeline([('scaler', MinMaxScaler()), 
                           ('LR', LogisticRegression(solver="liblinear", C=0.1, random_state=42))])

def acc_reg(*samples):
    samples = np.asarray(samples)
    
    X = samples[:-1].T
    y = samples[-1]
    
    if sum(y)==0 or sum(y)==len(y):
        return (999)
    
    test=[]
    preds=[]
    # 10 different training/test data splitting each bootstrapped replicate
    # (for a larger variety of potential output values)
    # Approximately a 80/20 split.
    for i in range(10):
        y_train=np.zeros(17)
        j=0
        while sum(y_train)==0 or sum(y_train)==len(y_train):
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.19, random_state=42*i+j)
            j+=1
        clf = rpipe.fit(X_train, y_train)
        pred_case=clf.predict(X_test)
        preds.extend(pred_case)
        test.extend(y_test)
      
    avs = 0
    for i in range(len(test)):  
        avs += abs(test[i] - preds[i])
    avs /= len(test)  # Normalize by the actual number of test samples

    return avs 

In [22]:
#Preparing the filtered dataset for model evaluation.

Xall=f_K27_RNA.to_numpy()
Yall=np.array(rDP)

samplesX = tuple(Xall[:, i] for i in range(Xall.shape[1]))
samples = samplesX + (Yall,)

In [23]:
#Bootstrapping for general accuracy.

res_acc = sts.bootstrap(samples, statistic=acc_reg, method='BCa', n_resamples=999, paired=True, random_state=24602)

In [24]:
res_acc.confidence_interval

ConfidenceInterval(low=np.float64(0.0), high=np.float64(0.225))

In [25]:
np.mean(res_acc.bootstrap_distribution)

np.float64(0.0916166166166166)

In [26]:
#Function for within-sample RO curve.

def roc_reg(*samples):
    samples = np.asarray(samples)
    
    X = samples[:-1].T
    y = samples[-1]
    
    if sum(y)==0 or sum(y)==len(y):
        return ('error: only one outcome present')
    
    test=[]
    preds=[]
    # 25 different training/test data splitting each bootstrapped replicate
    # (for a smoother ROC)
    # Approximately a 80/20 split.
    for i in range(25):
        y_train=np.zeros(17)
        j=0
        while sum(y_train)==0 or sum(y_train)==len(y_train):
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.19, random_state=42*i+j)
            j+=1
        
        clf = rpipe.fit(X_train, y_train)
        pred_case=clf.predict_proba(X_test)
        preds.extend(pred_case[:,1])
        test.extend(y_test)
      
    fpr, tpr, thres=sk.metrics.roc_curve(test,preds, drop_intermediate=False)
    auc=sk.metrics.auc(fpr, tpr)
    
    return auc

In [27]:
#Bootstrapping the ROC.

res_acc_roc = sts.bootstrap(samples, statistic=roc_reg, method='BCa', n_resamples=999, paired=True, random_state=24602)

In [28]:
res_acc_roc.confidence_interval

ConfidenceInterval(low=np.float64(0.8646289911640707), high=np.float64(1.0))

In [29]:
np.mean(res_acc_roc.bootstrap_distribution)

np.float64(0.951432115594679)

.

Given the within sample decent results of the Logistic Regression model, we train a full model with all the samples, to be evaluated in neuronal differentiation assays of Jerber et al., 2021.

.

In [30]:
#Build final model with all datapoints
# More aggressive regularisation (C=0.1) than the models for coefficient distributions, to prevent overfitting.
lgfinal = rpipe.fit(Xall, Yall)

In [31]:

#Importing Jerber data and preparing

RNA_hipsci_lines=pd.read_csv('hipsci-cell-lines.csv')[['Name']]

Jerber_assessed=pd.read_csv('Jerber2021_Lines_assessed.csv') #Supplementary Table 5 from Jerber et al., 2021.
Jerber_assessed=Jerber_assessed.rename(columns={"Predicted neuronal differentiation efficiency scores for all HipSci lines": "Name",
                                "Unnamed: 1": "Assessed", "Unnamed: 3": "diff_efficiency"})
Jerber_assessed=Jerber_assessed[(Jerber_assessed.Assessed=="succeeded") | (Jerber_assessed.Assessed=="failed")][['Name', 'Assessed', "diff_efficiency"]] 

Jerber_assessed["Simple_Name"]=Jerber_assessed.Name.str[-6:]

In [33]:
assessed_RNA=pd.merge(Jerber_assessed, RNA_hipsci_lines, on='Name')
simple_names=assessed_RNA.Simple_Name
assessed_RNA['bin_outc']= np.where(assessed_RNA['Assessed']=="succeeded", 0, 1)


In [34]:
# Which transcripts correspond to which gene
tx_per_gene=pd.read_csv("tx_per_gene_annot.csv", index_col=0)
tx_per_gene=tx_per_gene.rename(columns={"gene_id": "geneid"})

#Extracting the H3K27me3-related genes that we will use
txs_geneid_K27=tx_per_gene[tx_per_gene.geneid.isin(geneid_K27)]

In [38]:
#Extracting the data for each line of the hiPSCi consortium evaluated

for line in simple_names:
    line_tx_rna=pd.read_csv('hipsci_RNA/'+line+'.tsv', sep='\t')
    line_tx_rna=line_tx_rna.rename(columns={'target_id':'tx_id', 'tpm':line})
    line_tx_rna=line_tx_rna[["tx_id", line]]
    txs_geneid_K27=pd.merge(txs_geneid_K27, line_tx_rna, how='left', on="tx_id")


In [39]:
tpm_geneid_K27=txs_geneid_K27.groupby('geneid').sum().T
tpm_geneid_K27=tpm_geneid_K27.drop('tx_id')
tpm_geneid_K27

geneid,ENSG00000006016,ENSG00000006128,ENSG00000007372,ENSG00000009709,ENSG00000020181,ENSG00000026025,ENSG00000053438,ENSG00000054598,ENSG00000069812,ENSG00000075290,...,ENSG00000221818,ENSG00000229637,ENSG00000240065,ENSG00000242419,ENSG00000251493,ENSG00000266074,ENSG00000267909,ENSG00000273604,ENSG00000275023,ENSG00000279692
bezi_3,11.48354,1.865195,5.06321,0.266086,0.440241,67.329042,21.279,0.0,0.339006,0.842338,...,0.16852,0.0,0.804757,3.24847,0.0,0.370026,3.02352,0.0,0.0,0.0
eipl_1,27.009553,0.95091,2.49406,2.554693,3.03734,261.220034,56.0406,0.0,1.542355,0.213957,...,0.789508,0.0,0.269188,5.21783,0.0,10.074797,1.09617,0.0,0.0,0.0
fikt_3,15.45622,3.622563,0.445648,0.399028,0.86773,93.552177,0.231482,0.0,0.289233,0.731079,...,0.047499,0.0,0.062495,0.765246,0.0,0.118689,1.1195,0.0,0.0,0.0
kolf_2,19.712415,2.3945,0.363361,0.041248,1.224884,105.176115,5.07725,0.0,0.145655,0.346591,...,0.207377,0.0,0.375875,0.690142,0.0,0.245171,0.217274,0.0,0.0,0.0
lexy_2,7.296219,7.470543,0.221669,0.02758,0.40621,103.795458,9.38721,0.0,0.258448,0.094544,...,0.161349,0.0,0.418977,1.66561,0.0,0.064798,1.95364,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
vuud_2,16.209361,2.060189,0.439879,0.084199,0.657517,275.556802,6.97204,0.0,0.942007,1.02357,...,0.256942,0.0,0.285222,1.57265,0.0,0.412919,0.349669,0.0,0.0,0.0
ualf_6,6.97647,1.568818,3.434297,0.921646,1.615601,334.236699,10.5833,0.0,0.775464,0.749146,...,0.189891,0.0,0.381784,0.957502,0.153071,0.622421,1.01292,0.0,0.0,0.0
zuuy_4,7.491568,6.351459,21.043698,0.649588,0.692396,371.138546,34.7209,0.0,0.461433,5.1257,...,0.316928,0.0,0.401155,0.978905,0.0,0.632603,0.748772,0.0,0.0,0.0
hehd_1,15.0898,2.381057,0.52003,0.244798,1.451619,298.517157,5.40671,0.0,1.102763,0.964561,...,0.51391,0.0,0.097943,0.770983,0.0,0.426031,0.665853,0.0,0.0,0.0


In [40]:
# Columns that I'm retaining for model prediction
tpm_geneid_K27_c5=tpm_geneid_K27[f_K27_RNA.columns]

In [41]:
#Bootstrap the test for DNs

#Preparing the DN dataset for model evaluation.

XDN=tpm_geneid_K27_c5.to_numpy()
YDN=assessed_RNA.bin_outc.values

samplesX = tuple(XDN[:, i] for i in range(XDN.shape[1]))
samples = samplesX + (YDN,)

In [42]:
#Function for within-sample RO curve.

def prc_DN_mPGCLC(*samples):
    samples = np.asarray(samples)
    
    X = samples[:-1].T
    y = samples[-1]
    
    preds=lgfinal.predict_proba(X)

    prec, rec, thres=sk.metrics.precision_recall_curve(y.astype(int),preds[:,1])
    auc=sk.metrics.auc(rec, prec)
    return auc

In [43]:
#Function for within-sample RO curve.

def avs_DN_mPGCLC(*samples):
    samples = np.asarray(samples)
    
    X = samples[:-1].T
    y = samples[-1]
    
    preds=lgfinal.predict(X)

    avs = 0
    for i in range(len(preds)):  
        avs += abs(y[i] - preds[i])
    avs = avs/len(preds)  # Normalize by the actual number of test samples

    return avs 

In [44]:
res_acc_roc = sts.bootstrap(samples, statistic=avs_DN_mPGCLC, method='BCa', n_resamples=999, paired=True, random_state=24602)


In [45]:
res_acc_roc.confidence_interval

ConfidenceInterval(low=np.float64(0.14685314685314685), high=np.float64(0.2727272727272727))

In [46]:
np.mean(res_acc_roc.bootstrap_distribution)

np.float64(0.20204820204820206)

In [47]:
#Function for recall at 100% precision.

def rec100prec_DN_mPGCLC(*samples):
    samples = np.asarray(samples)
    
    X = samples[:-1].T
    y = samples[-1]
    
    preds=lgfinal.predict_proba(X)

    prec, rec, thres=sk.metrics.precision_recall_curve(y.astype(int),preds[:,1])
    for i in range(len(prec)):
        if prec[i]==1.0:
            rec100=rec[i]
            break
        
    return rec100

res_rec100 = sts.bootstrap(samples, statistic=rec100prec_DN_mPGCLC, method='BCa', n_resamples=9999, paired=True, random_state=24602)


In [48]:
res_rec100.confidence_interval

ConfidenceInterval(low=np.float64(0.14285714285714285), high=np.float64(0.46511627906976744))

In [49]:
np.mean(res_rec100.bootstrap_distribution)

np.float64(0.35738114964058193)

In [50]:
#Genes used for PGCLC prediction
pd.DataFrame(tpm_geneid_K27_c5.columns.values).to_csv("genes_PGCLC_pred.csv")